In [2]:
from utils import *
import tf_utils

from tensorflow_addons.metrics import CohenKappa
from tensorflow.keras.models import Model, load_model
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, BatchNormalization, Activation, concatenate, GlobalAveragePooling2D, Dense, ZeroPadding2D, AveragePooling2D

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import Nadam, SGD

import tensorflow as tf

from padding_generator import generator1, generator2

import os

import math
import pandas as pd
pd.set_option('display.max_colwidth', -1)

Forcing single GPU
Left next 1 GPU(s) unmasked: [3] (from [3 2] available)


/home/jr/anaconda3/envs/bap/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [3]:
#load data
data = load_data_from_csv(SetType.TRAIN)
data['Y'] = data['label'].apply(lambda x: 1 if x == 'True' else 0)

val_data = load_data_from_csv(SetType.VALID)
val_data['Y'] = val_data['label'].apply(lambda x: 1 if x == 'True' else 0)

In [5]:
#select models
selected_models = ['resnet_34', 'resnet_101', 'densenet']

#load models
models = [load_model(data_paths(m)['best'], compile=False) for m in selected_models]

model_count = len(selected_models)
for model, name in zip(models, selected_models):
    assert model.input.shape.as_list() == models[0].input.shape.as_list()
    
    #keras requires unique names
    model._name = name
        
input_size = model.input.shape.as_list()[1]

# create a model with n outputs
inp = Input((input_size, input_size, 1))
out = concatenate( [ model(inp) for model in models ] )
model_ens = Model(inputs=inp, outputs=out, name='Ensemble')

#predict data
print('Predicting...')
#this might take a while
pred = predict_all(model_ens, SetType.TRAIN, generator=generator2)
print('Predicting validation data...')
val_pred = predict_all(model_ens, SetType.VALID, generator=generator1)

Predicting...
Found 36256 validated image filenames belonging to 2 classes.
Found 36256 validated image filenames belonging to 2 classes.
Predicting validation data...
Found 3197 validated image filenames belonging to 2 classes.
Found 29 validated image filenames belonging to 2 classes.


In [7]:
#create model for stacking ensemble
inp_per = Input(len(models))
x = Dense(25, activation='relu')(inp_per)
x = Dense(25, activation='relu')(x)
out_per = Dense(1, activation='sigmoid', name = 'out')(x)

#checkpoint best
tmp_file_name = os.path.join(MODEL_FOLDER, 'tmp_best_perc.h5')
check = ModelCheckpoint(tmp_file_name, monitor='val_cohen_kappa', mode='max', verbose=1, save_best_only=True)

#create and fit model
p = Model(inputs=inp_per, outputs=out_per)
p.compile(SGD(0.1), loss='binary_crossentropy', metrics = CohenKappa(num_classes=2))
p.fit(pred, data['Y'].to_numpy(), validation_data = (val_pred, val_data['Y'].to_numpy()), epochs = 5, callbacks = [ check ], batch_size = 128)

#load best model
p = load_model(tmp_file_name)
os.remove(tmp_file_name)

Epoch 1/5
270/284 [===========================>..] - ETA: 0s - loss: 0.3758 - cohen_kappa: 0.6991
Epoch 00001: val_cohen_kappa improved from -inf to 0.61281, saving model to ../models/tmp_best_perc.h5
284/284 [==============================] - 1s 4ms/step - loss: 0.3732 - cohen_kappa: 0.6999 - val_loss: 0.4568 - val_cohen_kappa: 0.6128
Epoch 2/5
273/284 [===========================>..] - ETA: 0s - loss: 0.3105 - cohen_kappa: 0.7265
Epoch 00002: val_cohen_kappa did not improve from 0.61281
284/284 [==============================] - 1s 3ms/step - loss: 0.3107 - cohen_kappa: 0.7267 - val_loss: 0.4687 - val_cohen_kappa: 0.6031
Epoch 3/5
272/284 [===========================>..] - ETA: 0s - loss: 0.3085 - cohen_kappa: 0.7283
Epoch 00003: val_cohen_kappa did not improve from 0.61281
284/284 [==============================] - 1s 3ms/step - loss: 0.3092 - cohen_kappa: 0.7279 - val_loss: 0.4859 - val_cohen_kappa: 0.5996
Epoch 4/5
273/284 [===========================>..] - ETA: 0s - loss: 0.3093 

In [8]:
# unite and save
PATHS = data_paths('ensemble')

inp = Input((input_size, input_size, 1))

x = model_ens(inp)

model = Model(inputs=inp, outputs=p(x))
model.save(PATHS['best'])